This is jupyter notebook for test training runs

In [13]:
import pandas as pd
import numpy as np
from FNN import FNN
from gradient_descent import *
from ActivFunctions import  *
from LossFunctions import *

In [14]:
path_data = "data\wdbc.data"#data from https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic
regGoal = "radius1"

In [15]:
epochs = 1000
learning_rate = 0.001
batch_size = 24
loss_derivative = MeanSquaredErrorDerivative

In [16]:
columnNames = ["ID number","Diagnosis","radius1","texture1","perimeter1","area1","smoothness1","compactness1","concavity1","concave points1","symmetry1","fractal dimension1","radius2","texture2","perimeter2","area2","smoothness2","compactness2","concavity2","concave points2","symmetry2","fractal dimension2","radius3","texture3","perimeter3","area3","smoothness3","compactness3","concavity3","concave points3","symmetry3","fractal dimension3"]
data = pd.read_csv(path_data,names = columnNames)
data = data.drop(columns="ID number")#no need for id

In [17]:
#
data = data.drop(columns="Diagnosis")#categorical variable is unneeded
data = data.iloc[:,0:10]#using only mean features
dataInput = data.drop(columns=regGoal).to_numpy()
dataTarget = data.loc[:,regGoal].to_numpy()
#
dataInput = dataInput.T
dataTarget = dataTarget[np.newaxis,:]

In [18]:
#declaring network
net = FNN([9,64, 32, 16, 8, 4, 2, 1],
          [relu,identity],
          method_ini="HeUni"
)

In [19]:
n_samples = dataTarget.size
indices = np.arange(0,n_samples)
indices_test = np.random.permutation(round(n_samples*0.2))
indices_train = list(set(indices) - set(indices_test))

In [20]:
#
dataInput_train = dataInput[:,indices_train]
dataTarget_train = dataTarget[:,indices_train]
#
dataInput_test = dataInput[:,indices_test]
dataTarget_test = dataTarget[:,indices_test]
#
meanDataInput_train = np.mean(dataInput_train, axis=1)
meanDataInput_train = meanDataInput_train[:,np.newaxis]
stdDataInput_train = np.std(dataInput_train, axis=1)
stdDataInput_train = stdDataInput_train[:,np.newaxis]
dataInput_train = (dataInput_train - meanDataInput_train) / stdDataInput_train
dataInput_test =  (dataInput_test - meanDataInput_train) / stdDataInput_train
print(dataInput_train.shape)
print(dataInput_test.shape)

(9, 455)
(9, 114)


In [21]:
net_trained = train_minibatch_sgd(net, dataInput_train, dataTarget_train, epochs, learning_rate, batch_size, loss_derivative, grad_clip = 2)

In [22]:
out = net_trained.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

12.209890322859687


In [23]:
net_trained_adam = train_minibatch_adam(net,dataInput_train,dataTarget_train,epochs,learning_rate,batch_size,loss_derivative,beta1=0.9,beta2=0.999,epsilon=1e-8)

In [24]:
out = net_trained_adam.forward(dataInput_test)
predict = out[-1]
error = MeanSquaredError(dataTarget_test,predict[-1])
print(error)

2.1932414812608005
